In [1]:
import os, sys
import shutil
import pandas as pd
import numpy as np

In [2]:
main = os.getcwd()
print('main: ', main)
os.chdir(main + '\\' + 'tests_ignore')
prj = os.getcwd()
print('prj: ', prj)
# os.chdir(main)
# os.getcwd()

main:  c:\Users\jgoss\Documents\_Olin\DRF_engineering\LibGen_KiCad
prj:  c:\Users\jgoss\Documents\_Olin\DRF_engineering\LibGen_KiCad\tests_ignore


In [3]:
for root, dirs, files in os.walk(prj):
    print(root)
    print(dirs)
    print(files)

c:\Users\jgoss\Documents\_Olin\DRF_engineering\LibGen_KiCad\tests_ignore
[]
['ECG_MOUSE', 'ECG_MOUSE.lib', 'test_copy']


In [4]:
shutil.copy2('ECG_MOUSE', 'test_copy')
for root, dirs, files in os.walk(prj):
    print(files)

['ECG_MOUSE', 'ECG_MOUSE.lib', 'test_copy']


### Extract data from file into DataFrame

In [5]:
# python.dataScience.textOutputLimit = 0
with open('ECG_MOUSE', 'r') as schemtxt:
    data = schemtxt.read().splitlines()
    headers = data[0].split(', ')
    for line in range(0, len(data)):
        data[line] = data[line].split('","')
    data = data[1:len(data)]
    # print(len(data[0]))
    # print(data)

    df = pd.DataFrame(data, index = np.arange(1, len(data)+1), columns = headers)
    # print(df)


### Clean Dataframe Content